# Web Scrapping Comparis

In [35]:
import os
import math
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from getpass import getpass
import csv
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd


# 1. Get the links for all flats announced in City X

In [36]:
# select city and define url
city = 'Zuerich'.lower()
url = "https://www.comparis.ch/immobilien/marktplatz/"+city+"/wohnung/mieten?page=0"
url

'https://www.comparis.ch/immobilien/marktplatz/zuerich/wohnung/mieten?page=0'

In [37]:
wdpath = "chromedriver"
driver = webdriver.Chrome(wdpath)

## 1.1 Webscrapping for page 1

In [38]:
url = "https://www.comparis.ch/immobilien/marktplatz/"+city+"/wohnung/mieten?page=0"
driver.get(url)

In [39]:
## minimize the size of the side, so that we get all the links
driver.execute_script("document.body.style.zoom = '0.1'")

In [40]:
flats_link = driver.find_elements_by_class_name("excbu0j3")

In [41]:
flat_links = list(set([flats.get_attribute('href') for flats in flats_link]))
flat_links

['',
 'https://www.comparis.ch/immobilien/result/list?requestobject=%7B%22DealType%22%3A10%2C%22SiteId%22%3A0%2C%22RootPropertyTypes%22%3A%5B1%5D%2C%22PropertyTypes%22%3Anull%2C%22RoomsFrom%22%3A%223%22%2C%22RoomsTo%22%3A%223.5%22%2C%22FloorSearchType%22%3A0%2C%22LivingSpaceFrom%22%3Anull%2C%22LivingSpaceTo%22%3Anull%2C%22PriceFrom%22%3Anull%2C%22PriceTo%22%3A%221200%22%2C%22ComparisPointsMin%22%3A0%2C%22AdAgeMax%22%3A0%2C%22AdAgeInHoursMax%22%3Anull%2C%22Keyword%22%3A%22%22%2C%22WithImagesOnly%22%3Anull%2C%22WithPointsOnly%22%3Anull%2C%22Radius%22%3Anull%2C%22MinAvailableDate%22%3A%221753-01-01T00%3A00%3A00%22%2C%22MinChangeDate%22%3A%221753-01-01T00%3A00%3A00%22%2C%22LocationSearchString%22%3A%22Z%C3%BCrich%22%2C%22Sort%22%3A11%2C%22HasBalcony%22%3Afalse%2C%22HasTerrace%22%3Afalse%2C%22HasFireplace%22%3Afalse%2C%22HasDishwasher%22%3Afalse%2C%22HasWashingMachine%22%3Afalse%2C%22HasLift%22%3Afalse%2C%22HasParking%22%3Afalse%2C%22PetsAllowed%22%3Afalse%2C%22MinergieCertified%22%3Afalse%

In [42]:
links = list(set([x if len(str(re.findall('https://www.comparis.ch/immobilien/marktplatz/details/show/', str(x))))>=3 else 0 for x in flat_links]))[1:]

links

['https://www.comparis.ch/immobilien/marktplatz/details/show/23026064',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23025958',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23025554',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23025557',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23025556',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23025959',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23026014',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23025969',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23025678',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23025970']

In [43]:
print(len(links))

10


## 1.2. Get number of result pages

In [44]:
#amount of links
nr = list(set([x[-3:] if len(str(re.findall('page', str(x))))>=3 else 0 for x in flat_links]))[0:]


nrlis = ["".join(re.findall('[0-9]', str(x))) for x in nr]
resultpages=int(max(nrlis))
resultpages


99

## 1.3 Get links from flats from all the result pages

In [45]:
urlOhnePage = url[:-1]
for i in range(1, (resultpages-1)):
    page_url = urlOhnePage + str(i)
    driver.get(page_url)
    driver.execute_script("document.body.style.zoom = '0.1'")
    time.sleep(2.342)
    flats_link = driver.find_elements_by_class_name("excbu0j3")
    flat_links = list(set([flats.get_attribute('href') for flats in flats_link]))
    lin= list(set([x if len(str(re.findall('https://www.comparis.ch/immobilien/marktplatz/details/show/', str(x))))>=3 else 0 for x in flat_links]))[1:]
    for x in lin: 
        links.append(x)

## 1.4 Save lis of links to a csv

In [46]:
len(links)

970

In [47]:
linklist = pd.DataFrame(links, columns =['link'])
linklist.to_csv('./Data/links_'+city+'.csv')

# 2. Detailed Infos for each flat 

## 2.1 Load Links and change type to list

In [48]:
linklist = pd.read_csv('./Data/links_'+city+'.csv',  index_col=0)

In [49]:
linklist=list(linklist['link'])

In [50]:
len(linklist)

970

## 2.2 Infos for one flat

#### 2.2.0 get soup

In [51]:
url2 = linklist[0]

html2 = requests.get(url2).content
html2

soup2 = BeautifulSoup(html2, "lxml")  # Check documentation to specify 'features'
soup2

<!DOCTYPE html>
<html lang="De">
<head>
<title>4½ Zimmer Wohnung in 8047 Zürich ZH  – Immobilienmarkt von comparis.ch</title>
<meta content="follow,noindex" name="robots"/>
<meta content="Wohnung, 4½ Zimmer Wohnung, Miete, 8047 Zürich ZH " name="keywords"/>
<meta content="4½ Zimmer Wohnung in 8047 Zürich ZH , Mietpreis / Monat 3530 CHF  " name="description"/>
<meta content="comparis.ch AG" name="copyright"/>
<meta content="de" http-equiv="Content-Language"/>
<meta content="de" name="language"/>
<meta content="private" http-equiv="Cache-Control"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/><script type="text/javascript">window.NREUM||(NREUM={});NREUM.info = {"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"30d2eb686c","applicationID":"63661895","transactionName":"NAYHNkEFVhUCVUQICw1MJDFjS20PKlhEBAMRAhELXApwBw1SXAQW","queueTime":0,"applicationTime":526,"agent":"","atts":""}</script><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config=

In [52]:
nr = int(url2[-8:])
nr

23026064

In [53]:
source =pd.DataFrame.from_dict({'city': city, 'DateofScrapping': '26.02.2020'}, orient='index', columns = [nr] )
source

23026064
city                zuerich
DateofScrapping  26.02.2020

#### 2.2.1  Adress and Date of Publication

In [54]:
###

"""
<h3 class="text-green">Riedbachstrasse 80, 3027 Bern</h3>

"""
adress_soup = soup2.find_all('h3', attrs = {'class':'text-green'})
if len(adress_soup)==0:
    adress = {'adress': 'NaN'}
else:
    adress = {'adress': adress_soup[0].text}
adress
adress = pd.DataFrame.from_dict(adress, orient='index', columns = [nr] )
adress

23026064
adress  8047 Zürich ZH

In [55]:
#Date of Publication

"""
<div class="row xsmall-12 columns">
        <ul class="meta-tag-line">
            <li>
                Erstmals gefunden am 26.02.2020
            </li>
        </ul>
    </div>
"""

publication={}
publication_soup = soup2.find_all('div', attrs = {'class':'row xsmall-12 columns'})
for element in publication_soup:
    for x in element.find_all('li'):
        publication['publication_date']=(x.text.replace('\r\n','').replace('Erstmals gefunden am ','').replace('\r\n','').replace(" ",''))
publication
publication_date = pd.DataFrame.from_dict(publication, orient='index', columns = [nr] )
publication_date

23026064
publication_date  26.02.2020

#### (##2.2.2  Overview (First large table)##)

In [56]:
## Overview from one flat --> not used?
overview=[]
overview_soup = soup2.find_all('div', attrs = {'class':'item-price large'})
for element in overview_soup:
    for x in element.find_all('strong'):
        overview.append(x.text)
overview


['Wohnung', "CHF 3'530", '4½', 'ca. CHF 445 /Jahr']

#### 2.2.3 Ratings

In [57]:
#### rating
"""
<div class="info-grid-content">
                    <span class="item-rating medium">
                        <strong>
                            <i class="text-green icon-comparis-points" data-icon="data-icon">
</i> 4
                        </strong>
                    </span>
                </div>
"""
rating_soup = soup2.find_all('span', attrs = {'class':'item-rating medium'})
if len(rating_soup)==0:
    rating = {'rating': 'NaN'}
else:
    rating = {'rating': re.findall('[0-9]',rating_soup[0].text)[0]}
rating
rating = pd.DataFrame.from_dict(rating, orient='index', columns = [nr] )
rating

23026064
rating      NaN

In [58]:
rating_soup

[]

#### 2.2.4 Detailed overview (second smaller table)

In [59]:
infos_soup = soup2.find_all('dl')

### Categorie name
info1=[]
for x in infos_soup[0].find_all('dt'):
    info1.append(x.text.replace(" ","").replace("\r\n",""))

info1

### actual info

info2=[]
for x in infos_soup[0].find_all('dd'):
    info2.append(x.text.replace('\r\n','').replace('CHF','').replace(' ','').replace('\n\n', '1').replace("'","").replace("m²","").replace('½','.5'))

info2

info={}
for x in range(len(info2)):
    info[info1[x]] = info2[x]

info = pd.DataFrame.from_dict(info, orient='index', columns = [nr] )
info


23026064
Objektart                   Wohnung
MietpreisproMonat              3530
Zimmer                          4.5
Verfügbarab        nachVereinbarung

##### 2.6 to one column

In [60]:
ds=pd.concat([info, rating, adress, publication_date, source])

In [61]:
ds = ds.transpose()
ds

Objektart MietpreisproMonat Zimmer       Verfügbarab rating  \
23026064   Wohnung              3530    4.5  nachVereinbarung    NaN   

                  adress publication_date     city DateofScrapping  
23026064  8047 Zürich ZH       26.02.2020  zuerich      26.02.2020

# 3. Repeat for other flats

In [62]:
## repeat in a loop with all url
for x in linklist:
    print(x)


https://www.comparis.ch/immobilien/marktplatz/details/show/23026064
https://www.comparis.ch/immobilien/marktplatz/details/show/23025958
https://www.comparis.ch/immobilien/marktplatz/details/show/23025554
https://www.comparis.ch/immobilien/marktplatz/details/show/23025557
https://www.comparis.ch/immobilien/marktplatz/details/show/23025556
https://www.comparis.ch/immobilien/marktplatz/details/show/23025959
https://www.comparis.ch/immobilien/marktplatz/details/show/23026014
https://www.comparis.ch/immobilien/marktplatz/details/show/23025969
https://www.comparis.ch/immobilien/marktplatz/details/show/23025678
https://www.comparis.ch/immobilien/marktplatz/details/show/23025970
https://www.comparis.ch/immobilien/marktplatz/details/show/23025154
https://www.comparis.ch/immobilien/marktplatz/details/show/23024860
https://www.comparis.ch/immobilien/marktplatz/details/show/23025749
https://www.comparis.ch/immobilien/marktplatz/details/show/23024828
https://www.comparis.ch/immobilien/marktplatz/de

In [63]:
link=linklist[1:]
link

['https://www.comparis.ch/immobilien/marktplatz/details/show/23025958',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23025554',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23025557',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23025556',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23025959',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23026014',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23025969',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23025678',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23025970',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23025154',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23024860',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23025749',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23024828',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/230

In [69]:
## repeat in a loop with all url
link = link[858:]
for x in link:
    #print(x)
    # get soup (2.1):
    html2 = requests.get(x).content
    soup2 = BeautifulSoup(html2, "lxml") 
    # get id:
    nr = int(x[-8:])
    ## city and date of webscrapping
    source =pd.DataFrame.from_dict({'city': city, 'DateofScrapping': '26.02.2020'}, orient='index', columns = [nr] )
    
    # adress:
    adress_soup = soup2.find_all('h3', attrs = {'class':'text-green'})
    if len(adress_soup)==0:
        adress = {'adress': 'NaN'}
    else:
        adress = {'adress': adress_soup[0].text}
    adress = pd.DataFrame.from_dict(adress, orient='index', columns = [nr] )

    #publication Date:
    publication={}
    publication_soup = soup2.find_all('div', attrs = {'class':'row xsmall-12 columns'})
    for element in publication_soup:
        for x in element.find_all('li'):
            publication['publication_date']=(x.text.replace('\r\n','').replace('Erstmals gefunden am ','').replace('\r\n','').replace(" ",''))
    publication_date = pd.DataFrame.from_dict(publication, orient='index', columns = [nr] )
    
    
    # rating: 
    rating_soup = soup2.find_all('span', attrs = {'class':'item-rating medium'})
    if len(rating_soup)==0:
        rating = {'rating': 'NaN'}
    else:
        rating = {'rating': re.findall('[0-9]',rating_soup[0].text)[0]}
    rating = pd.DataFrame.from_dict(rating, orient='index', columns = [nr] )

    
    # details: 
    infos_soup = soup2.find_all('dl')
    # Categorie name
    info1=[]
    for x in infos_soup[0].find_all('dt'):
        info1.append(x.text.replace(" ","").replace("\r\n",""))
    # actual info
    info2=[]
    for x in infos_soup[0].find_all('dd'):
        info2.append(x.text.replace('\r\n','').replace('CHF','').replace(' ','').replace('\n\n', '1').replace("'","").replace("m²","").replace('½','.5'))
    info={}
    for x in range(len(info2)):
        info[info1[x]] = info2[x]
    info = pd.DataFrame.from_dict(info, orient='index', columns = [nr] )
    
    #row
    row =pd.concat([info, rating, adress, publication_date, source])
    row = row.transpose()
    # merge with dataset
    ds=pd.concat([ds, row], sort=False)
    print(ds.shape[0])

849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959


In [74]:
ds

Objektart MietpreisproMonat Zimmer       Verfügbarab rating  \
23026064   Wohnung              3530    4.5  nachVereinbarung    NaN   
23025958   Wohnung              1125      1  nachVereinbarung    NaN   
23025554   Wohnung              1200      1        01.05.2020    NaN   
23025557   Wohnung              3000    3.5  nachVereinbarung    NaN   
23025556   Wohnung              1650      2  nachVereinbarung    NaN   
...            ...               ...    ...               ...    ...   
22901785   Wohnung              5830    5.5        01.03.2020    NaN   
22701512   Wohnung              1700    2.5            sofort    NaN   
22701488   Wohnung              2510    3.5        01.04.2020      5   
22949170   Wohnung              1785    1.5        01.04.2020    NaN   
22967884   Wohnung              2800    2.5        01.04.2020    NaN   

                                       adress publication_date     city  \
23026064                       8047 Zürich ZH       26.02.2020  zuerich   
23025958                       8052 Zürich ZH       26.02.2020  zuerich   
23025554  Limmattalstrasse 71, 8049 Zürich ZH       26.02.2020  zuerich   
23025557                       8057 Zürich ZH       26.02.2020  zuerich   
23025556                       8004 Zürich ZH       26.02.2020  zuerich   
...                                       ...              ...      ...   
22901785                          8038 Zürich       08.01.2020  zuerich   
22701512          Schürgistr. 71, 8051 Zürich                   zuerich   
22701488    Oberwiesenstrasse 67, 8050 Zürich                   zuerich   
22949170             Nägelihof 3, 8001 Zürich       08.01.2020  zuerich   
22967884    Restelbergstrasse 26, 8044 Zürich       07.01.2020  zuerich   

         DateofScrapping Wohnfläche  ... Geschirrspülmaschine Wasseranschluss  \
23026064      26.02.2020        NaN  ...                  NaN             NaN   
23025958      26.02.2020        NaN  ...                  NaN             NaN   
23025554      26.02.2020         25  ...                  NaN             NaN   
23025557      26.02.2020        NaN  ...                  NaN             NaN   
23025556      26.02.2020        NaN  ...                  NaN             NaN   
...                  ...        ...  ...                  ...             ...   
22901785      26.02.2020        151  ...                  NaN             NaN   
22701512      26.02.2020         80  ...                  NaN             NaN   
22701488      26.02.2020         81  ...                  NaN             NaN   
22949170      26.02.2020         32  ...                  NaN             NaN   
22967884      26.02.2020         80  ...                  NaN             NaN   

         Kubatur Eckhaus Veranda Hochparterre Hebebühne Anfahrrampe Holzofen  \
23026064     NaN     NaN     NaN          NaN       NaN         NaN      NaN   
23025958     NaN     NaN     NaN          NaN       NaN         NaN      NaN   
23025554     NaN     NaN     NaN          NaN       NaN         NaN      NaN   
23025557     NaN     NaN     NaN          NaN       NaN         NaN      NaN   
23025556     NaN     NaN     NaN          NaN       NaN         NaN      NaN   
...          ...     ...     ...          ...       ...         ...      ...   
22901785     NaN     NaN     NaN          NaN       NaN         NaN      NaN   
22701512     NaN     NaN     NaN          NaN       NaN         NaN      NaN   
22701488     NaN     NaN     NaN          NaN       NaN         NaN      NaN   
22949170     NaN     NaN     NaN          NaN       NaN         NaN      NaN   
22967884     NaN     NaN     NaN          NaN       NaN         NaN      NaN   

         Grundstückfläche  
23026064              NaN  
23025958              NaN  
23025554              NaN  
23025557              NaN  
23025556              NaN  
...                   ...  
22901785              NaN  
22701512              NaN  
22701488              NaN  
22949170              NaN  
2

## 3.3 Save to csv

In [71]:
ds.to_csv('./Data/flats_'+ city + '.csv')

In [72]:
ds.columns

Index(['Objektart', 'MietpreisproMonat', 'Zimmer', 'Verfügbarab', 'rating',
       'adress', 'publication_date', 'city', 'DateofScrapping', 'Wohnfläche',
       'Geschoss', 'Kabel-TV', 'Rollstuhlgängig', 'Baujahr', 'Balkon(e)',
       'Einstellplatz', 'Aussenparkplatz', 'Cheminée',
       'MietpreisproMonat(exkl.NK)', 'Nebenkosten', 'Lift', 'ÖV-Haltestelle',
       'Einkaufsmöglichkeit', 'Haustiereerlaubt', 'Autobahn', 'Kindergarten',
       'Primarschule', 'Sekundarschule', 'Raumhöhe', 'Wohnungen', 'Bäder',
       'Neubau', 'Still', 'Sonnig', 'Nutzfläche', 'Toiletten',
       'Kinderfreundlich', 'Erstbezug', 'Minergie-zertifiziert',
       'AltesGebäude', 'Geschirrspülmaschine', 'Wasseranschluss', 'Kubatur',
       'Eckhaus', 'Veranda', 'Hochparterre', 'Hebebühne', 'Anfahrrampe',
       'Holzofen', 'Grundstückfläche'],
      dtype='object')

In [73]:
ds.shape[0]

959

#### 